In [3]:
nums = [1, 2, 3, 4]

for num in nums:
    print(num)

print('---')

index = 0
while True:
    try:
        num = nums.__getitem__(index)
        print(num)
        index += 1
    except IndexError:
        break


1
2
3
4
---
1
2
3
4


In [18]:
class Sequence:
    def __init__(self, length):
        self.length = length

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        if index >= len(self) or index < -len(self):
            raise IndexError
        return index


s = Sequence(5)

print(s[0])
# print(s[100]) --> IndexError
# print(s[-100]) --> IndexError

print('---')

for index in s:
    print(index)

print('---')

for index in s:
    print(index)


0
---
0
1
2
3
4
---
0
1
2
3
4


In [15]:
print(type(slice(None, None, -1)))
print(slice(0, 5).indices(3))
print(slice(0, 5).indices(8))
print(type(slice(0, 5).indices(8)))

print(list(range(*slice(0, 5).indices(8))))


<class 'slice'>
(0, 3, 1)
(0, 5, 1)
<class 'tuple'>
[0, 1, 2, 3, 4]


In [18]:
from functools import lru_cache


class Fib:
    def __init__(self, n):
        self.n = n

    @staticmethod
    @lru_cache(2 ** 10)
    def fib(n):
        if n < 2: return 1
        return Fib.fib(n - 1) + Fib.fib(n - 2)

    def __len__(self):
        return self.n
    
    def __getitem__(self, index):
        if isinstance(index, int):
            if index < 0:
                index += len(self)
            if index < 0 or index >= len(self):
                raise IndexError
            return Fib.fib(index)
        else:
            # slice.indicies(length) --> tuple
            # list(range(tuple))
            return [Fib.fib(value) for value in range(*index.indices(len(self)))]
    

f = Fib(10)

print(list(f))
print(f[-1])

print(f[0:5])

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
55
[1, 1, 2, 3, 5]
